In [1]:
import pandas as pd
import openpyxl #for reading or saving to Excel
import xlsxwriter
import sys

import state_country_dicts as scd
import requests
from bs4 import BeautifulSoup as bs
# import regular expressions
import re 
import time

In [2]:
def clean_up (number_str):
    '''This is a clean up routine so that only numeric characters remain'''
    number_str = number_str.replace(',', '').replace('null','0').rstrip(']').lstrip("'")
    return int(number_str)

In [3]:
pwd

'/Users/edwardterrell/Desktop/covid_machinations'

In [4]:
# The start dates for U.S.states and for countries are different so must be accounted for
# st_start = pd.Timestamp('2020/3/13')
c_start = pd.Timestamp('2020/2/16')

# set end time to yesterday

#day == pd.Timestamp.today().strftime('%Y-%m-%d')
end = pd.Timestamp.today() - pd.Timedelta(days=1)
print(f'end changed to yesterday: {end}')
#else:
#   end = pd.Timestamp.today()
#    print(f'end changed to {end}')
    
# set the index to the appropriate date range for states and countries
#st_index = pd.date_range(st_start, end)
c_index = pd.date_range(c_start, end)
#len(st_index)
len(c_index)

# list comprehension to extract just the state initials from the dict
#st_list = list(state_dict.values())

# create an empty pandas df with column headers from state_dict
#state = pd.DataFrame(columns = st_list,index=st_index)
#state.sort_index(ascending = False).head()
#state.shape

end changed to yesterday: 2022-01-01 12:45:02.545027


686

In [5]:
# creates a custom country dict
c_string = input('Enter country codes seperated by a space:') or 'fr it es'
clist = scd.custom_list (c_string)

country_dict = {key: value for key, value in scd.all_countries.items() if value in clist}
print(country_dict)

print()

Enter country codes seperated by a space: 


{'spain': 'ES', 'france': 'FR', 'italy': 'IT'}



In [6]:
c_list = list(country_dict.values())
c_list

['ES', 'FR', 'IT']

In [7]:
# create an empty pandas df with column headers from country_dict, using the starting dates for the country as index
country = pd.DataFrame(columns = c_list,index=c_index)
country.sort_index(ascending = False).head()

,ES,FR,IT
2022-01-01,NaN,NaN,NaN
2021-12-31,NaN,NaN,NaN
2021-12-30,NaN,NaN,NaN
2021-12-29,NaN,NaN,NaN
2021-12-28,NaN,NaN,NaN


In [8]:
base_url='https://www.worldometers.info/coronavirus/country/'
# Here the keys are the country names and the values are the 2-letter abbreviation
for key, value in country_dict.items():
    url = base_url+key
    print(url)

    # scrape web page for country info and assign to soup
    response = requests.get(url)
    #response.status_code
    page = response.text
    #print(response.text[:400])
    soup = bs(page)
    
    # Find block of text/data near 'graph-cases-daily' 
    re_graph = re.compile('graph-cases')
    dat = soup.find(text=re_graph)
   
    try:
        data = soup.find(text=re_graph).parent
    except:
        print(f'Unable to find {url}')
        print()
        break

    # Regex to pull out data chunk out of the larger soup like data
    myregex = re.compile (r'data:.*\]')
    mo = myregex.search(str(data))

    #mo.group() is a string so easy to get rid of the first entry
    country_previous_cases = mo.group().split(',')[1:]
    #print(country_previous_cases)
    previous_cases = ["0" if i == 'null' else i for i in country_previous_cases]
    
    len(previous_cases)

    # merge countries into the existing df and apply the function clean_up to convert strings to ints
    country[value] = previous_cases
    country[value] = country[value].fillna(0).apply(clean_up)
    
    time.sleep(2)
country.clip(lower=0,inplace=True) #large negative number removed for better graphic clarity
country.sort_index(ascending = False).head()

https://www.worldometers.info/coronavirus/country/spain
https://www.worldometers.info/coronavirus/country/france
https://www.worldometers.info/coronavirus/country/italy


,ES,FR,IT
2022-01-01,0,219126,141353
2021-12-31,0,232200,144237
2021-12-30,161688,206243,126900
2021-12-29,100760,208099,98142
2021-12-28,99671,179807,78299


### Scrape Section

In [225]:
url='https://www.worldometers.info/coronavirus/'
# scrape web page for table
response = requests.get(url)
print(response.status_code)
page = response.text
soup = bs(page)

200


In [243]:
country_href = [item for item in soup.find('tbody').find_all(class_='mt_a')]
country_names = [item.text for item in soup.find('tbody').find_all(class_='mt_a')]
print (f'country_names is length {len(country_names)}')

country_names is length 221


In [244]:
#population = [item.text for item in soup.find('tbody').find_all(class_='mt_a')]

In [245]:
country_names[:10]

['USA',
 'India',
 'Brazil',
 'UK',
 'Russia',
 'France',
 'Turkey',
 'Germany',
 'Spain',
 'Italy']

In [246]:
country_href[:5]

[<a class="mt_a" href="country/us/">USA</a>,
 <a class="mt_a" href="country/india/">India</a>,
 <a class="mt_a" href="country/brazil/">Brazil</a>,
 <a class="mt_a" href="country/uk/">UK</a>,
 <a class="mt_a" href="country/russia/">Russia</a>]

In [247]:
#soup.find('tbody').find_all('a')

In [248]:
world_list = [item for item in soup.find('tbody').find_all('a')]
print(len(world_list))
world_list[:5]

439


[<a class="mt_a" href="country/us/">USA</a>,
 <a href="/world-population/us-population/">333,918,903</a>,
 <a class="mt_a" href="country/india/">India</a>,
 <a href="/world-population/india-population/">1,400,383,713</a>,
 <a class="mt_a" href="country/brazil/">Brazil</a>]

In [249]:
for item in world_list[0:5]:
    print(item['href'])

country/us/
/world-population/us-population/
country/india/
/world-population/india-population/
country/brazil/


In [250]:
pop_list=[]
country_list=[]
for item in world_list:
    if "population" in item['href']: 
        pop_list.append(item.text)
        country_list.append(item['href'][18:-12])

In [251]:
zip_it = zip(country_list,pop_list)
country_pop = dict(zip_it)
print (len(country_pop))

218


[('saint-kitts-and-nevis', '53,764'),
 ('bhutan', '784,369'),
 ('greenland', '56,918'),
 ('saint-barthelemy', '9,922'),
 ('anguilla', '15,204'),
 ('wallis-and-futuna-islands', '10,943'),
 ('saint-pierre-and-miquelon', '5,751'),
 ('falkland-islands-malvinas', '3,632'),
 ('china-macao-sar', '662,626'),
 ('montserrat', '4,997'),
 ('holy-see', '804'),
 ('solomon-islands', '712,151'),
 ('palau', '18,224'),
 ('western-sahara', '619,368'),
 ('vanuatu', '317,912'),
 ('marshall-islands', '59,790'),
 ('samoa', '200,395'),
 ('saint-helena', '6,104'),
 ('micronesia', '116,834'),
 ('tonga', '107,494')]

<span style="background-color:lightgreen">There are 4 countries that are not assigned a population</span>

In [ ]:
# We will address these later
country_names.remove('Israel')
country_names.remove ('Qatar')
country_names.remove ('New Zealand')
country_names.remove ('China')

In [257]:
zip_names = zip(country_list,country_names)
country_country = dict(zip_names)
print (len(country_country))
#list(country_country.items())[0:5]
#list(country_pop.items())[:5]

In [265]:
def clean_up (number_str):
    '''This is a clean up routine so that only numeric characters remain'''
    number_str = number_str.replace(',', '').replace('null','0').rstrip(']').lstrip("'")
    return int(number_str)

pop_dict = {k: clean_up (v) for k, v in country_pop.items()}

In [272]:
list(country_country.items())[0:5]

[('us', 'USA'),
 ('india', 'India'),
 ('brazil', 'Brazil'),
 ('uk', 'UK'),
 ('russia', 'Russia')]

In [267]:
list(pop_dict.items())[:5]

[('us', 333918903),
 ('india', 1400383713),
 ('brazil', 214829399),
 ('uk', 68421816),
 ('russia', 146028505)]

In [301]:
url = 'https://www.infoplease.com/us/states/state-population-by-rank'
tables = pd.read_html(url)
state_pop = tables[0].iloc[:-1,1:3]
state_pop.columns=('State','pop_millions')
state_pop.pop_millions = state_pop.pop_millions/1000000
state_pop.head()

,State,pop_millions
0,California,39.538223
1,Texas,29.145505
2,Florida,21.538187
3,New York,20.201249
4,Pennsylvania,13.002700


In [309]:
df = pd.DataFrame(list(us_states.items()))
df.columns = ('State', 'Code')
df.head()

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [312]:
state_population = pd.merge(df,state_pop,on='State')
state_population.head()

,State,Code,pop_millions
0,Alabama,AL,5.024279
1,Alaska,AK,0.733391
2,Arizona,AZ,7.151502
3,Arkansas,AR,3.011524
4,California,CA,39.538223


In [317]:
state_population.to_csv('test.csv',index=False)

In [318]:
df = pd.read_csv('test.csv')
df

,State,Code,pop_millions
0,Alabama,AL,5.024279
1,Alaska,AK,0.733391
2,Arizona,AZ,7.151502
3,Arkansas,AR,3.011524
4,California,CA,39.538223
5,Colorado,CO,5.773714
6,Connecticut,CT,3.605944
7,Delaware,DE,0.989948
8,Florida,FL,21.538187
9,Georgia,GA,10.711908


In [302]:
# translate States to Two-Letter codes
us_states = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}



In [ ]:
!pwd

In [ ]:
def polish_report_total(df):
    '''polish_report removes and reorders columns and assigns shortened names. It also
    creates a datetime index, changes strings to integers and handles NaNs
    Returns: a copy of the original df
    '''
    # eliminate unnecessary columns and re-order columns
    df = df.iloc[:,[-2,0,3,6,7]].copy()
    
    # shorten col names for easier manipulation
    df.columns = ['total','name','type', 'county','date']
    
    # use the date column as the index
    df.index = df.date
    df.drop('date',axis=1, inplace=True)
    
    # replace nuisance commas
    df.name = df.name.str.replace(',','_')
    
    return df

In [ ]:
# create dfs for both active and resolved sheets 
for i in range(2):
    i=i+1
    #df = pd.read_excel(f'covid_data/covid_report_{current_report}.xlsx',sheet_name=i,parse_dates=['Date reported to public health'])
    df = pd.read_excel('~/Desktop/covid-19-data/covid_data/covid_report_2021-12-22.xlsx',sheet_name=i,parse_dates=['Date reported to public health'])
    df = df.iloc[:,:16]
    df = polish_report_total(df)
    df = df.sort_index(ascending=False)
    if i == 1: 
        df1 = df.iloc[:,:4]
    elif i == 2: 
        df2 = df.iloc[:,:4]

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
# Merge both dfs into a single and change all types to integers
totals = pd.merge(df1,df2,left_index=True, right_index=True,how='outer' ).fillna(0)
#totals = totals.astype(int)
totals.head()

In [ ]:
totals.shape

In [ ]:
#pd.merge?

In [ ]:
# Merge both dfs into a single and change all types to integers
totals = df1.merge(df2)
#totals = totals.astype(int)
totals.head()

In [ ]:
# Create a column which sums active and resolved totals
totals['all_reported_cases'] = totals.total_x+totals.total_y

# Create a Date column for plotting purposes 
# Keep only the last two columns
totals['Date'] = totals.index.strftime('%Y-%m')
totals = totals.iloc[:,-2:]
totals

In [ ]:
totals[['Date','all_reported_cases']].plot.bar(figsize=(10,6), \
                                x='Date',rot=45, width=.8, \
                                color="red",alpha=.7,\
                                title="Outbreaks (total new cases) in Colorado per month");

### More Detailed Inquiry

In [ ]:
# create dfs for both active and resolved sheets 
for i in range(2):
    i=i+1
    #df = pd.read_excel(f'covid_data/covid_report_{current_report}.xlsx',sheet_name=i,parse_dates=['Date reported to public health'])
    df = pd.read_excel('~/Desktop/covid-19-data/covid_data/covid_report_2021-12-22.xlsx',sheet_name=i,parse_dates=['Date reported to public health'])
    df = df.iloc[:,:16]
    df = polish_report_total(df)
    
    
    df = df.sort_index(ascending=False)
    if i == 1: 
        active = df
    elif i == 2: 
        resolved = df
        

In [20]:
# Test code below
url ='https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
tables = pd.read_html(url)
tables[1].tail()

,Presidency[a],Presidency[a].1,Portrait,Name(Birth–Death),Party[b],Party[b].1,Election,Vice President
78,44,"January 20, 2009–January 20, 2017",NaN,Barack Obama(b. 1961),NaN,Democratic,2008,Joe Biden
79,44,"January 20, 2009–January 20, 2017",NaN,Barack Obama(b. 1961),NaN,Democratic,2012,Joe Biden
80,45,"January 20, 2017–January 20, 2021",NaN,Donald Trump(b. 1946),NaN,Republican,2016,Mike Pence
81,46,"January 20, 2021–Incumbent",NaN,Joe Biden(b. 1942),NaN,Democratic,2020,Kamala Harris
82,Sources:[15][16][17],Sources:[15][16][17],Sources:[15][16][17],Sources:[15][16][17],Sources:[15][16][17],Sources:[15][16][17],Sources:[15][16][17],Sources:[15][16][17]
